In [49]:
spark

In [50]:
import hashlib
from datetime import datetime
from graphframes import *
from pyspark.sql import functions as func
from pyspark.sql.types import FloatType
import hsfs
from hops import hdfs
import os
from pyspark.sql import SQLContext

In [51]:
def hashnode(x):
    return hashlib.sha1(x.encode("UTF-8")).hexdigest()[:8]

hashnode_udf = func.udf(hashnode)

In [52]:
# Create a connection
connection = hsfs.connection()
# Get the feature store handle for the project's feature store
fs = connection.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

In [53]:
edge_fg = fs.get_feature_group('transactions_fg', 1)
node_fg = fs.get_feature_group('account_features', 1)

In [54]:
node_fg.show(5)

An error was encountered:
An error occurred while calling o446.showString.
: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.SparkContext.getOrCreate(SparkContext.scala)
org.apache.livy.rsc.driver.SparkEntries.sc(SparkEntries.java:53)
org.apache.livy.rsc.driver.SparkEntries.sparkSession(SparkEntries.java:67)
org.apache.livy.repl.AbstractSparkInterpreter.postStart(AbstractSparkInterpreter.scala:69)
org.apache.livy.repl.SparkInterpreter$$anonfun$start$1.apply$mcV$sp(SparkInterpreter.scala:88)
org.apache.livy.repl.SparkInterpreter$$anonfun$start$1.apply(SparkInterpreter.scala:63)
org.apache.livy.repl.SparkInterpreter$$anonfun$start$1.apply(SparkInterpreter.scala:63)
org.apache.livy.repl.AbstractSparkInterpreter.restoreContextClassLoader(AbstractSparkInterpreter.scala:340)
org.apache.livy.repl.SparkInterpreter.start(SparkInterpreter.scala:63)
org.apache.livy.repl.Session$$anonfun$1.apply(Session.sca

In [55]:
edge_fg.show(5)

An error was encountered:
An error occurred while calling o490.showString.
: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.SparkContext.getOrCreate(SparkContext.scala)
org.apache.livy.rsc.driver.SparkEntries.sc(SparkEntries.java:53)
org.apache.livy.rsc.driver.SparkEntries.sparkSession(SparkEntries.java:67)
org.apache.livy.repl.AbstractSparkInterpreter.postStart(AbstractSparkInterpreter.scala:69)
org.apache.livy.repl.SparkInterpreter$$anonfun$start$1.apply$mcV$sp(SparkInterpreter.scala:88)
org.apache.livy.repl.SparkInterpreter$$anonfun$start$1.apply(SparkInterpreter.scala:63)
org.apache.livy.repl.SparkInterpreter$$anonfun$start$1.apply(SparkInterpreter.scala:63)
org.apache.livy.repl.AbstractSparkInterpreter.restoreContextClassLoader(AbstractSparkInterpreter.scala:340)
org.apache.livy.repl.SparkInterpreter.start(SparkInterpreter.scala:63)
org.apache.livy.repl.Session$$anonfun$1.apply(Session.sca

In [56]:
only_sar_edge_df = edge_fg.read().filter(func.col('alert_id') != -1)
only_normal_edge_df =  edge_fg.read().filter(func.col('alert_id') == -1)

In [57]:
only_normal_edge_df.show(5)

An error was encountered:
An error occurred while calling o541.showString.
: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.SparkContext.getOrCreate(SparkContext.scala)
org.apache.livy.rsc.driver.SparkEntries.sc(SparkEntries.java:53)
org.apache.livy.rsc.driver.SparkEntries.sparkSession(SparkEntries.java:67)
org.apache.livy.repl.AbstractSparkInterpreter.postStart(AbstractSparkInterpreter.scala:69)
org.apache.livy.repl.SparkInterpreter$$anonfun$start$1.apply$mcV$sp(SparkInterpreter.scala:88)
org.apache.livy.repl.SparkInterpreter$$anonfun$start$1.apply(SparkInterpreter.scala:63)
org.apache.livy.repl.SparkInterpreter$$anonfun$start$1.apply(SparkInterpreter.scala:63)
org.apache.livy.repl.AbstractSparkInterpreter.restoreContextClassLoader(AbstractSparkInterpreter.scala:340)
org.apache.livy.repl.SparkInterpreter.start(SparkInterpreter.scala:63)
org.apache.livy.repl.Session$$anonfun$1.apply(Session.sca

In [58]:
only_sar_edge_df.show()

An error was encountered:
An error occurred while calling o536.showString.
: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.SparkContext.getOrCreate(SparkContext.scala)
org.apache.livy.rsc.driver.SparkEntries.sc(SparkEntries.java:53)
org.apache.livy.rsc.driver.SparkEntries.sparkSession(SparkEntries.java:67)
org.apache.livy.repl.AbstractSparkInterpreter.postStart(AbstractSparkInterpreter.scala:69)
org.apache.livy.repl.SparkInterpreter$$anonfun$start$1.apply$mcV$sp(SparkInterpreter.scala:88)
org.apache.livy.repl.SparkInterpreter$$anonfun$start$1.apply(SparkInterpreter.scala:63)
org.apache.livy.repl.SparkInterpreter$$anonfun$start$1.apply(SparkInterpreter.scala:63)
org.apache.livy.repl.AbstractSparkInterpreter.restoreContextClassLoader(AbstractSparkInterpreter.scala:340)
org.apache.livy.repl.SparkInterpreter.start(SparkInterpreter.scala:63)
org.apache.livy.repl.Session$$anonfun$1.apply(Session.sca

In [ ]:
only_sar_edge_df.count()

In [12]:
sar_sources = only_sar_edge_df.select("source")
sar_targets = only_sar_edge_df.select("target")
sar_nodes = sar_sources.union(sar_targets).toDF("id").dropDuplicates()
sar_nodes.count()

sar_edges = only_sar_edge_df.select("source", "target").toDF("src", "dst")
sar_edges.count()

732

In [13]:
# Now lets construct the graph
g_sar = GraphFrame(sar_nodes,sar_edges)
sc.setCheckpointDir("hdfs:///Projects/{}/Logs/sc".format(hdfs.project_name()))
cc_sar = g_sar.connectedComponents()

In [14]:
cc_sar.cache().show()

+----+---------+
|  id|component|
+----+---------+
|5300|     2773|
| 463|      124|
|3997|     2773|
|8086|     2599|
|7833|     3671|
|1127|     1127|
| 540|      540|
|6393|     3396|
|5614|      825|
|1522|     1054|
|3488|     2599|
|2393|      397|
|9162|     2889|
|7387|      643|
|1265|     1009|
|4042|     1590|
|5223|     1013|
|4364|     1096|
|3425|     1054|
|5157|       61|
+----+---------+
only showing top 20 rows

In [15]:
cc_sar.groupBy('component').count().select('count').dropDuplicates().orderBy('count').show()

+-----+
|count|
+-----+
|    5|
|    6|
|    7|
|    8|
|    9|
|   10|
+-----+

In [16]:
scc_sar = g_sar.stronglyConnectedComponents(20)

In [17]:
scc_sar.cache().show()

+----+---------+
|  id|component|
+----+---------+
|7800|     7800|
|8600|     8600|
|4000|     4000|
|5201|     1109|
|1801|     1801|
| 601|      601|
|1201|     1201|
|4601|     4601|
|9601|     9601|
|3402|     3402|
|4802|     4802|
|7802|     1945|
|6403|     1630|
|1003|     1003|
|2003|     1201|
|5403|      295|
|9403|     9403|
|7604|     1201|
|5204|     1257|
|8004|     8004|
+----+---------+
only showing top 20 rows

In [18]:
scc_sar.groupBy('component').count().select('count').dropDuplicates().orderBy('count').show()

+-----+
|count|
+-----+
|    1|
|    5|
|    6|
|    7|
|    8|
|    9|
|   10|
+-----+

In [19]:
scc_sar.groupBy('component').count().where(func.col('count')==1).count()

471

In [20]:
scc_sar.groupBy('component').count().where(func.col('count')>1).count()

40

In [21]:
scc_comp_count = scc_sar.groupBy('component').count().where(func.col('count')>1)

In [22]:
scc_sar = scc_sar.join(scc_comp_count,['component'])

In [23]:
scc_sar.show()

+---------+----+-----+
|component|  id|count|
+---------+----+-----+
|     1152|3595|   10|
|     1152|1152|   10|
|     1152|8535|   10|
|     1152|4324|   10|
|     1152|2321|   10|
|     1152|8117|   10|
|     1152|3304|   10|
|     1152|8654|   10|
|     1152|8049|   10|
|     1152|7824|   10|
|      399| 399|    6|
|      399|8965|    6|
|      399|9960|    6|
|      399|7146|    6|
|      399|2501|    6|
|      399|5621|    6|
|     3671|4969|    6|
|     3671|8546|    6|
|     3671|3671|    6|
|     3671|4867|    6|
+---------+----+-----+
only showing top 20 rows

In [24]:
scc_sar =  scc_sar.drop('count')

In [25]:
sar_cc_grouped = cc_sar.join(
    only_sar_edge_df,
    [(only_sar_edge_df.source==cc_sar.id)|(only_sar_edge_df.target==cc_sar.id)],
    how="left"
).dropDuplicates(subset=['tran_id'])

In [26]:
sar_scc_grouped = scc_sar.join(
    only_sar_edge_df,
    [(only_sar_edge_df.source==scc_sar.id)|(only_sar_edge_df.target==scc_sar.id)],
    how="left"
).dropDuplicates(subset=['tran_id'])

In [27]:
sar_cc_grouped.count()

732

In [28]:
sar_scc_grouped.count()

321

In [29]:
only_sar_edge_df.count()

732

In [30]:
sar_scc_grouped.show()

+---------+----+-------+------+-------+--------------+--------+------+--------+------+
|component|  id|tx_type|is_sar|tran_id|tran_timestamp|base_amt|target|alert_id|source|
+---------+----+-------+------+-------+--------------+--------+------+--------+------+
|     1009|1425|      4|     1| 100274|   1.4892768E9|  102.86|  1425|      80|  3661|
|     1801|9700|      4|     1| 284874|   1.5004224E9|  103.47|  3984|      96|  9700|
|      554|8091|      4|     1| 522545|   1.5147648E9|  106.94|  8091|      91|   554|
|      825|4571|      4|     1| 706588|    1.525824E9|   63.66|  7776|      84|  4571|
|     1369|1598|      4|     1| 472208|   1.5117408E9|  114.67|  8581|      70|  1598|
|     1257|5261|      4|     1|  57885|   1.4866848E9|   73.41|  6837|      95|  5261|
|      907|7851|      4|     1| 509573|   1.5139872E9|   47.28|  3329|      90|  7851|
|      397|3252|      4|     1| 786028|   1.5306624E9|   79.42|  6415|      74|  3252|
|      295|6810|      4|     1| 202624|   1

In [31]:
only_sar_edge_df = sar_scc_grouped

In [32]:
only_sar_edge_df.show(5)

+---------+----+-------+------+-------+--------------+--------+------+--------+------+
|component|  id|tx_type|is_sar|tran_id|tran_timestamp|base_amt|target|alert_id|source|
+---------+----+-------+------+-------+--------------+--------+------+--------+------+
|     1009|3661|      4|     1| 100274|   1.4892768E9|  102.86|  1425|      80|  3661|
|     1801|3984|      4|     1| 284874|   1.5004224E9|  103.47|  3984|      96|  9700|
|      554|8091|      4|     1| 522545|   1.5147648E9|  106.94|  8091|      91|   554|
|      825|4571|      4|     1| 706588|    1.525824E9|   63.66|  7776|      84|  4571|
|     1369|8581|      4|     1| 472208|   1.5117408E9|  114.67|  8581|      70|  1598|
+---------+----+-------+------+-------+--------------+--------+------+--------+------+
only showing top 5 rows

In [33]:
only_normal_edge_df.count()

1028964

In [34]:
only_sar_edge_df_grouped = only_sar_edge_df.groupBy('component').agg(func.min("tran_timestamp"),func.max("tran_timestamp")).toDF("component", "window_start", "window_end")
only_sar_edge_df_grouped.show(5)

+---------+------------+-----------+
|component|window_start| window_end|
+---------+------------+-----------+
|     1152|  1.516752E9|1.5170976E9|
|      399| 1.5412032E9|1.5428448E9|
|     3671| 1.5287616E9| 1.530144E9|
|     3751| 1.4901408E9|1.4911776E9|
|     1945| 1.5290208E9|1.5299712E9|
+---------+------------+-----------+
only showing top 5 rows

In [35]:
only_sar_edges_df_windows = only_sar_edge_df.join(only_sar_edge_df_grouped,["component"])

In [36]:
only_sar_edges_df_windows.show()

+---------+----+-------+------+-------+--------------+--------+------+--------+------+------------+-----------+
|component|  id|tx_type|is_sar|tran_id|tran_timestamp|base_amt|target|alert_id|source|window_start| window_end|
+---------+----+-------+------+-------+--------------+--------+------+--------+------+------------+-----------+
|     1152|8654|      4|     1| 561549|   1.5170976E9|    43.6|  8654|      65|  8535|  1.516752E9|1.5170976E9|
|     1152|3304|      4|     1| 559541|   1.5170112E9|   73.84|  3304|      65|  8049|  1.516752E9|1.5170976E9|
|     1152|4324|      4|     1| 561547|   1.5170976E9|   53.83|  2321|      65|  4324|  1.516752E9|1.5170976E9|
|     1152|1152|      4|     1| 556095|    1.516752E9|  101.29|  8535|      65|  1152|  1.516752E9|1.5170976E9|
|     1152|8049|      4|     1| 561548|   1.5170976E9|   48.44|  8049|      65|  2321|  1.516752E9|1.5170976E9|
|     1152|3304|      4|     1| 559542|   1.5170112E9|   66.45|  8117|      65|  3304|  1.516752E9|1.517

In [37]:
only_sar_edges_df_windows.count()

321

In [38]:
only_normal_edges_df_windows = only_sar_edge_df_grouped.select("window_start", "window_end").join(
    only_normal_edge_df,
    [(only_normal_edge_df.tran_timestamp>=only_sar_edge_df_grouped.window_start)&(only_normal_edge_df.tran_timestamp<=only_sar_edge_df_grouped.window_end)],
    how="left"
)

In [39]:
only_normal_edges_df_windows.show()

+------------+-----------+-------+------+-------+--------------+--------+------+--------+------+
|window_start| window_end|tx_type|is_sar|tran_id|tran_timestamp|base_amt|target|alert_id|source|
+------------+-----------+-------+------+-------+--------------+--------+------+--------+------+
|  1.516752E9|1.5170976E9|      4|     0| 555045|    1.516752E9|  503.89|  9947|      -1|  6249|
|  1.516752E9|1.5170976E9|      4|     0| 555046|    1.516752E9|  564.26|  3095|      -1|  4557|
|  1.516752E9|1.5170976E9|      4|     0| 555047|    1.516752E9| 7773.05|  3003|      -1|  4967|
|  1.516752E9|1.5170976E9|      4|     0| 555048|    1.516752E9|  4490.0|  9866|      -1|  7090|
|  1.516752E9|1.5170976E9|      4|     0| 555049|    1.516752E9| 6136.03|  1343|      -1|  4471|
|  1.516752E9|1.5170976E9|      4|     0| 555050|    1.516752E9| 9344.63|  3076|      -1|  6975|
|  1.516752E9|1.5170976E9|      4|     0| 555051|    1.516752E9| 1527.03|  1969|      -1|  6923|
|  1.516752E9|1.5170976E9|    

In [40]:
only_normal_edges_df_windows.count()

677241

In [41]:
only_normal_edges_df_windows = only_normal_edges_df_windows.withColumnRenamed("source", "origId")\
                                                           .withColumnRenamed("target", "destId")  
only_normal_edges_df_windows = only_normal_edges_df_windows.withColumn('target',hashnode_udf(func.concat(func.col('destId'),func.lit('_'),func.col('window_start'),func.lit('_'),func.col('window_end'))))\
                                                           .withColumn('source',hashnode_udf(func.concat(func.col('origId'),func.lit('_'),func.col('window_start'),func.lit('_'),func.col('window_end'))))            
only_normal_edges_df_windows.show()

+------------+-----------+-------+------+-------+--------------+--------+------+--------+------+--------+--------+
|window_start| window_end|tx_type|is_sar|tran_id|tran_timestamp|base_amt|destId|alert_id|origId|  target|  source|
+------------+-----------+-------+------+-------+--------------+--------+------+--------+------+--------+--------+
|  1.516752E9|1.5170976E9|      4|     0| 555045|    1.516752E9|  503.89|  9947|      -1|  6249|b786ed64|bbc43466|
|  1.516752E9|1.5170976E9|      4|     0| 555046|    1.516752E9|  564.26|  3095|      -1|  4557|b0b7a393|af529315|
|  1.516752E9|1.5170976E9|      4|     0| 555047|    1.516752E9| 7773.05|  3003|      -1|  4967|4156c046|e033872d|
|  1.516752E9|1.5170976E9|      4|     0| 555048|    1.516752E9|  4490.0|  9866|      -1|  7090|05ac06b4|3eac3624|
|  1.516752E9|1.5170976E9|      4|     0| 555049|    1.516752E9| 6136.03|  1343|      -1|  4471|e1901da0|cb468f43|
|  1.516752E9|1.5170976E9|      4|     0| 555050|    1.516752E9| 9344.63|  3076|

In [42]:
normal_sources = only_normal_edges_df_windows.select("source")
normal_targets = only_normal_edges_df_windows.select("target")
normal_nodes = normal_sources.union(normal_targets).toDF("id").dropDuplicates()
normal_edges = only_normal_edges_df_windows.select("source", "target").toDF("src", "dst")

In [43]:
# Now lets construct the graph
g_normal = GraphFrame(normal_nodes,normal_edges)

In [48]:
1-1

0

In [45]:
sc.setCheckpointDir("hdfs:///Projects/{}/Logs/sc".format(hdfs.project_name()))
#cc_normal = g_normal.connectedComponents()
scc_normal = g_normal.stronglyConnectedComponents(20).cache()

An error was encountered:
An error occurred while calling o418.run.
: java.lang.IllegalStateException: SparkContext has been shutdown
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2053)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2158)
	at org.apache.spark.rdd.RDD$$anonfun$fold$1.apply(RDD.scala:1098)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:363)
	at org.apache.spark.rdd.RDD.fold(RDD.scala:1092)
	at org.apache.spark.graphx.impl.VertexRDDImpl.count(VertexRDDImpl.scala:90)
	at org.apache.spark.graphx.GraphOps.numVertices$lzycompute(GraphOps.scala:41)
	at org.apache.spark.graphx.GraphOps.numVertices(GraphOps.scala:41)
	at org.apache.spark.graphx.lib.StronglyConnectedComponents$.run(StronglyConnectedComponents.scala:50)
	at org.graphframes.lib.StronglyConnectedComponents$.org$graphframes$l

In [46]:
scc_norm_comp_count = scc_normal.groupBy('component').count().where(func.col('count')>1)
scc_normal = scc_normal.join(scc_norm_comp_count,['component'])
scc_normal =  scc_normal.drop('count')
normal_scc_grouped = normal_normal.join(
    only_normal_edge_df,
    [(only_normal_edge_df.source==scc_normal.id)|(only_normal_edge_df.target==scc_normal.id)],
    how="left"
).dropDuplicates(subset=['tran_id'])

An error was encountered:
name 'scc_normal' is not defined
Traceback (most recent call last):
NameError: name 'scc_normal' is not defined



In [47]:
normal_scc_grouped.show()

An error was encountered:
name 'normal_scc_grouped' is not defined
Traceback (most recent call last):
NameError: name 'normal_scc_grouped' is not defined

